In [1]:
import numpy as np
from scipy.optimize import least_squares
#from pandas import Series, DataFrame
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
#
# if pade.py is not in the current directory, set this path:
#
#import sys
#sys.path.append('../Python_libs')
from rac_aux import *

In [2]:
Angs2Bohr=1.8897259886
au2eV=27.211386027
au2cm=219474.63068
#
#  files in the current directory do not need the path name
#
#df = pd.read_csv("/home/thomas/Python/StabPlots/Stab_data/1D_a0.2_b0_c0.14/crossing_1.dat", delim_whitespace=True)
df = pd.read_csv("sb_rac.csv")
#df = pd.read_csv("crossing_1.dat", delim_whitespace=True)

plot_it=False
if plot_it:
    plt.cla()
    plt.plot(df.l.values, df.E1.values, 'o-')
    plt.plot(df.l.values, df.E2.values, 'o-')
    plt.plot(df.l.values, df.E3.values, 'o-')
    plt.show()
df[:5]

,l,E1,E2,E3,E4
0,0.0,0.469219,1.365173,1.938043,3.271038
1,0.2,0.457846,1.314401,1.844238,3.242423
2,0.4,0.446073,1.251127,1.760926,3.215267
3,0.6,0.433754,1.173608,1.690292,3.189317
4,0.8,0.420648,1.082215,1.632448,3.164365


In [3]:
#
# put all negative E(lambda) points into the vectors: ls and Es
#
i_neg = np.argmin(abs(df.E1.values))
if df.E1[i_neg] > 0:
    i_neg += 1
ls = df.l.values[i_neg:]
print('N=',len(ls))
Es = df.E1.values[i_neg:]
if plot_it:
    plt.cla()
    plt.plot(df.l.values, df.E1.values, 'b-')
    plt.plot(df.l.values, df.E2.values, 'b-')
    plt.plot(df.l.values, df.E3.values, 'b-')
    plt.plot(ls, Es, 'o', color="orange")
    plt.show()

N= 89


In [15]:
#
#  So far, nm can be in [31, 42, 53]
#
nm=53

fun=pade_53_lsq
jac=pade_53j_lsq

In [16]:
#
#  kappas, kappa**2, and sigmas (weights = sigma**2)
#  least_squares() passes parg to each pade_nm function
#
k2s = -Es
ks  = np.sqrt(k2s)
sigmas = weights(len(Es), 'ones')
#sigmas = weights(len(Es), 'energy', E0=Es[11], Es=Es)
parg=(ks,k2s,ls,sigmas)

In [17]:
# start params depend on nm
p31_opt = [2.4022, 0.2713, 1.2813, 0.4543]
p42_opt = [2.3919, 0.2964, 1.3187, 1.3736,  0.29655, 0.5078]

E0 = linear_extra(ls,Es)
G0 = 0.2*E0
if nm == 31:
    p0s=[ls[0]] + guess(E0, G0) + [10]
elif nm == 42:
    p0s=[ls[0]] + guess(E0, G0) + guess(5*E0,10*G0) + [10]
elif nm == 53:
    p0s = p42_opt[0:5] + p31_opt[3:] + p42_opt[5:] + [1]
else:
    print("Warning", nm, "not implemented")
print(p0s)
print(chi2_gen(p0s, ks, k2s, ls, sigmas, fun))

[2.3919, 0.2964, 1.3187, 1.3736, 0.29655, 0.4543, 0.5078, 1]
33.16551690973274


In [18]:
#
#  test the derivative of [n,m] 
#

N=6

df1s = pade_gen_j_lsq(p0s, ks[-N:], k2s[-N:], ls[-N:], sigmas[-N:], fun)
print("num grad:\n", df1s)

df2s = jac(p0s, ks[-N:], k2s[-N:], ls[-N:], sigmas[-N:])
print("ana grad:\n", df2s)

np.sqrt(np.sum(np.square(df1s-df2s)))

num grad:
 [[ 0.38953879  0.19117545 -0.15721997  0.79000311  0.81109868  1.68043641
  -0.18612995 -1.33510246]
 [ 0.39132513  0.19145959 -0.1565389   0.79572221  0.81986972  1.6952413
  -0.18644192 -1.34579397]
 [ 0.39309983  0.19173764 -0.15586284  0.80138975  0.82862584  1.70997787
  -0.18674545 -1.35640547]
 [ 0.39486315  0.19200981 -0.15519185  0.8070067   0.83736726  1.72464725
  -0.18704083 -1.36693848]
 [ 0.39661534  0.19227628 -0.15452596  0.81257403  0.8460942   1.73925054
  -0.18732833 -1.3773945 ]
 [ 0.39835662  0.19253723 -0.1538652   0.81809265  0.85480687  1.75378883
  -0.18760822 -1.38777496]]
ana grad:
 [[ 0.38953879  0.19117545 -0.15721997  0.79000311  0.81109868  1.68043641
  -0.18612995 -1.33510246]
 [ 0.39132513  0.19145959 -0.1565389   0.79572221  0.81986972  1.6952413
  -0.18644192 -1.34579397]
 [ 0.39309983  0.19173764 -0.15586284  0.80138975  0.82862584  1.70997787
  -0.18674545 -1.35640547]
 [ 0.39486315  0.19200981 -0.15519185  0.8070067   0.83736726  1.72464

2.6316503427431527e-10

In [19]:
print('Least squares, trust-region-reflective (default) with 2-point jac')
res = least_squares(fun, p0s, method='trf', jac='2-point', args=parg)
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Least squares, trust-region-reflective (default) with 2-point jac
njev: 680
cost: 2.464708335230457e-10
grad: [-5.98547864e-07 -1.94619022e-07 -7.99501404e-08 -1.06433631e-07
 -2.13735827e-06 -2.23383567e-06  1.05170237e-06  1.46892149e-06]
message: The maximum number of function evaluations is exceeded.
success: False
x: [2.39254975 0.18738935 1.33775657 0.59494288 1.16928673 0.34302819
 1.53887509 0.63907868]
chi2 = 4.929e-10
Er=1.788360,  Gamma=0.187900


In [20]:
print('Least squares, trust-region-reflective (default) with analytic jac')
res = least_squares(fun, p0s, method='trf', jac=jac, args=parg)
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Least squares, trust-region-reflective (default) with analytic jac
njev: 680
cost: 2.4645400091317617e-10
grad: [-2.42933588e-06 -7.90156285e-07 -3.26027037e-07 -4.32162831e-07
 -8.66949422e-06 -9.05631022e-06  4.26980080e-06  5.95501098e-06]
message: The maximum number of function evaluations is exceeded.
success: False
x: [2.39254979 0.18741919 1.33774473 0.59490325 1.16941168 0.34301569
 1.53903683 0.63912608]
chi2 = 4.929e-10
Er=1.788327,  Gamma=0.187958


In [21]:
print('Least squares, Levenberg-Marquardt with analytic jac')
res = least_squares(fun, p0s, method='lm', jac=jac, args=parg)
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Least squares, Levenberg-Marquardt with analytic jac
njev: 731
cost: 4.887227848979291e-10
grad: [-4.15684838e-06 -9.86981100e-07 -8.24279629e-08 -9.89705148e-07
 -1.57495763e-05 -1.45261004e-05  7.74804179e-06  1.05477683e-05]
message: The maximum number of function evaluations is exceeded.
success: False
x: [2.39249781 0.14658027 1.3662456  0.6532054  1.00830421 0.36333732
 1.33436644 0.58071153]
chi2 = 9.774e-10
Er=1.866165,  Gamma=0.117419


In [22]:
print('Least squares, TRF with bounds')

#
#  bnds depend on the number of parameters
#
npara=len(p0s)
zs = np.zeros(npara)
infs = np.full(npara, np.inf)
bnds=(zs, infs)

res = least_squares(fun, p0s, jac=jac, bounds=bnds, args=parg)
print("njev:",res.njev)
print("cost:",res.cost)
print("grad:",res.grad)
print("message:",res.message)
print("success:",res.success)
print("x:", res.x)
print('chi2 = %.3e' % (res.cost*2))
print("Er=%f,  Gamma=%f" % res_ene(res.x[1], res.x[2]))

Least squares, TRF with bounds
njev: 615
cost: 7.961773505817201e-10
grad: [-9.40616555e-07 -2.37224725e-07  1.14828319e-07 -2.81137234e-07
 -3.90438027e-06 -3.42509822e-06  1.78589943e-06  2.69627545e-06]
message: The maximum number of function evaluations is exceeded.
success: False
x: [2.39245952 0.17944622 1.42298845 0.67319159 0.87472383 0.39277737
 1.17094572 0.54136252]
chi2 = 1.592e-09
Er=2.023859,  Gamma=0.183286


In [23]:
#
# swipe energy filter 
#
M=len(Es)
sigmas = weights(M, 'ones')
res = least_squares(fun, p0s, method='trf', jac=jac, args=(ks, k2s, ls, sigmas))
Er, G = res_ene(res.x[1], res.x[2])
print('All weights equal:  chi2 = %.3e  Er=%f,  Gamma=%f' % (res.cost*2, Er, G))
M=len(Es)
for n in [0, M//4, M//2, 3*M//4, M-1]:
    sigmas = weights(M, 'energy', E0=Es[n], Es=Es)
    res = least_squares(fun, p0s, method='trf', jac=jac, args=(ks, k2s, ls, sigmas))
    Er, G = res_ene(res.x[1], res.x[2])
    print('Filter E = %6.2f:  chi2 = %.3e  Er=%f,  Gamma=%f' % (Es[n], res.cost*2, Er, G))

All weights equal:  chi2 = 4.929e-10  Er=1.788327,  Gamma=0.187958
Filter E =  -0.00:  chi2 = 1.342e-11  Er=1.746616,  Gamma=0.324691
Filter E =  -3.30:  chi2 = 1.794e-11  Er=1.809663,  Gamma=0.399221
Filter E =  -6.81:  chi2 = 1.128e-11  Er=1.804075,  Gamma=0.412699
Filter E = -10.43:  chi2 = 7.942e-12  Er=1.802438,  Gamma=0.414644
Filter E = -14.11:  chi2 = 1.012e-11  Er=1.788863,  Gamma=0.437290
